In [46]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from termcolor import colored
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor



In [47]:
train=pd.read_excel(r"C:\Users\Nasem\Downloads\ViscosityPredOfEpoxyResin-main\ViscosityPredOfEpoxyResin-main\dataset for ML (after data cleaning).xlsx")


In [48]:
train.head()

,Temperature/℃,Epoxy Resin,Additional Group of Epoxy resin,Diluent,Additional Group of Diluent,Ratio of Epoxy Resin to Diluent,Viscosity
0,29.795191,bisphenol A,H,glycidyl ether,n-butyl,95：5,1243.068262
1,29.736358,bisphenol A,H,glycidyl ether,n-butyl,90：10,597.888811
2,29.731920,bisphenol A,H,glycidyl ether,n-butyl,85：15,368.191414
3,29.729081,bisphenol A,H,glycidyl ether,n-butyl,80：20,221.252639
4,34.840280,bisphenol A,H,glycidyl ether,n-butyl,95：5,776.917664


In [49]:
train.head()

,Temperature/℃,Epoxy Resin,Additional Group of Epoxy resin,Diluent,Additional Group of Diluent,Ratio of Epoxy Resin to Diluent,Viscosity
0,29.795191,bisphenol A,H,glycidyl ether,n-butyl,95：5,1243.068262
1,29.736358,bisphenol A,H,glycidyl ether,n-butyl,90：10,597.888811
2,29.731920,bisphenol A,H,glycidyl ether,n-butyl,85：15,368.191414
3,29.729081,bisphenol A,H,glycidyl ether,n-butyl,80：20,221.252639
4,34.840280,bisphenol A,H,glycidyl ether,n-butyl,95：5,776.917664


In [50]:
count = [col for col in train.columns if train[col].dtype == 'object']
count_values = [train[col].nunique() for col in count]

print("Categorical Columns:")
for col, count_value in zip(count, count_values):
    print(f"- {col}: {count_value}")

numeric_cols = [col for col in train.columns if train[col].dtype != 'object']

print("\nNumeric Columns:")
for col in numeric_cols:
    print(f"- {col}")

Categorical Columns:
- Epoxy Resin: 3
- Additional Group of Epoxy resin: 5
- Diluent: 10
- Additional Group of Diluent: 30
- Ratio of Epoxy Resin to Diluent: 40

Numeric Columns:
- Temperature/℃
- Viscosity


In [51]:
num_rows, num_cols = train.shape
print(f"DataFrame Dimensions:\nNumber of rows: {num_rows}\nNumber of columns: {num_cols}\n")
print("DataFrame Summary:")
train.info()

DataFrame Dimensions:
Number of rows: 999
Number of columns: 7

DataFrame Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Temperature/℃                    999 non-null    float64
 1   Epoxy Resin                      999 non-null    object 
 2   Additional Group of Epoxy resin  999 non-null    object 
 3   Diluent                          999 non-null    object 
 4   Additional Group of Diluent      999 non-null    object 
 5   Ratio of Epoxy Resin to Diluent  999 non-null    object 
 6   Viscosity                        999 non-null    float64
dtypes: float64(2), object(5)
memory usage: 54.8+ KB


In [52]:
desc = train.describe().T
desc['nunique'] = train.nunique()
desc['%unique'] = desc['nunique'] / len(train) * 100
desc['null'] = train.isna().sum()
desc['type'] = train.dtypes
desc.head(60)

,count,mean,std,min,25%,50%,75%,max,nunique,%unique,null,type
Temperature/℃,999.0,42.913470,17.720846,20.000000,25.236882,39.775647,55.000000,100.0,484,48.448448,0,float64
Viscosity,999.0,2180.655611,7286.814373,0.000071,35.514467,152.542373,980.386282,137200.0,969,96.996997,0,float64


In [53]:
def heatmap(dataset, title):
    
    corr = dataset.corr()
    fig, axes = plt.subplots(figsize=(20, 10))
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(corr, linewidths=0.5, mask=mask, cmap='plasma', annot=True, annot_kws={'size': 20})
    plt.title(title, fontsize=40)
    plt.xticks(fontsize=16, rotation=25)
    plt.yticks(fontsize=16, rotation=25)
    plt.show()

In [54]:
train.head()

,Temperature/℃,Epoxy Resin,Additional Group of Epoxy resin,Diluent,Additional Group of Diluent,Ratio of Epoxy Resin to Diluent,Viscosity
0,29.795191,bisphenol A,H,glycidyl ether,n-butyl,95：5,1243.068262
1,29.736358,bisphenol A,H,glycidyl ether,n-butyl,90：10,597.888811
2,29.731920,bisphenol A,H,glycidyl ether,n-butyl,85：15,368.191414
3,29.729081,bisphenol A,H,glycidyl ether,n-butyl,80：20,221.252639
4,34.840280,bisphenol A,H,glycidyl ether,n-butyl,95：5,776.917664


In [55]:
categorical_columns = ['Epoxy Resin', 'Additional Group of Epoxy resin', 'Diluent', 'Additional Group of Diluent']
train = pd.get_dummies(train, columns=categorical_columns)

In [56]:
train.head()

,Temperature/℃,Ratio of Epoxy Resin to Diluent,Viscosity,Epoxy Resin_bisphenol A,Epoxy Resin_ester cyclic,Epoxy Resin_phenolic,Additional Group of Epoxy resin_Amyl phenol acid,Additional Group of Epoxy resin_H,Additional Group of Epoxy resin_acrylate,Additional Group of Epoxy resin_acrylate,Additional Group of Epoxy resin_methacrylate,Diluent_Furan acetone,Diluent_Others,Diluent_acrylate,Diluent_epoxy oil,Diluent_ester,Diluent_glycidyl ether,Diluent_phenol,Diluent_phenolic,Diluent_polysiloxane,Diluent_siloxane,"Additional Group of Diluent_1, 4-butanediol","Additional Group of Diluent_1, 6-hexanediol",Additional Group of Diluent_Bisphosphonate-piperazine hydroxyl,Additional Group of Diluent_Cardanol was grafted with silicone,Additional Group of Diluent_Dipropylene glycol,Additional Group of Diluent_Epoxy ricin acid,Additional Group of Diluent_H,Additional Group of Diluent_Parsley methyl group,Additional Group of Diluent_Polyoxycardanol,Additional Group of Diluent_anacardol,Additional Group of Diluent_benzyl alcohol,Additional Group of Diluent_butyl,Additional Group of Diluent_cyclohexene oxide,Additional Group of Diluent_diglycol,Additional Group of Diluent_epoxy group,Additional Group of Diluent_eugenol,Additional Group of Diluent_heterocycle,Additional Group of Diluent_methylene,Additional Group of Diluent_methyl,Additional Group of Diluent_n-butyl,Additional Group of Diluent_nonyl,Additional Group of Diluent_o-xylene,Additional Group of Diluent_octyl group,Additional Group of Diluent_p-aminophenol,Additional Group of Diluent_phenyl,Additional Group of Diluent_phenylethylene oxide,Additional Group of Diluent_phosphoric acid,Additional Group of Diluent_propylene glycol,Additional Group of Diluent_styrene,Additional Group of Diluent_thymol
0,29.795191,95：5,1243.068262,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,29.736358,90：10,597.888811,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,29.731920,85：15,368.191414,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,29.729081,80：20,221.252639,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,34.840280,95：5,776.917664,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [57]:
print(len(train.columns))

51


In [58]:
train.columns

Index(['Temperature/℃', 'Ratio of Epoxy Resin to Diluent', 'Viscosity',
       'Epoxy Resin_bisphenol A', 'Epoxy Resin_ester cyclic',
       'Epoxy Resin_phenolic',
       'Additional Group of Epoxy resin_Amyl phenol acid  ',
       'Additional Group of Epoxy resin_H',
       'Additional Group of Epoxy resin_acrylate',
       'Additional Group of Epoxy resin_acrylate ',
       'Additional Group of Epoxy resin_methacrylate', 'Diluent_Furan acetone',
       'Diluent_Others', 'Diluent_acrylate', 'Diluent_epoxy oil',
       'Diluent_ester', 'Diluent_glycidyl ether', 'Diluent_phenol',
       'Diluent_phenolic', 'Diluent_polysiloxane', 'Diluent_siloxane ',
       'Additional Group of Diluent_1, 4-butanediol',
       'Additional Group of Diluent_1, 6-hexanediol',
       'Additional Group of Diluent_Bisphosphonate-piperazine hydroxyl',
       'Additional Group of Diluent_Cardanol was grafted with silicone',
       'Additional Group of Diluent_Dipropylene glycol',
       'Additional Group of Di

In [60]:
import pandas as pd
import re

# Assuming you already have a DataFrame named 'train'
# train = pd.DataFrame()  # Just an example, replace with your actual dataframe

# Function to rename features
def rename_feature(feature):
    # Remove any special characters
    clean_feature = re.sub(r'[^a-zA-Z0-9\s]', '', feature)
    # Convert spaces and double spaces to underscores and make lowercase
    return re.sub(r'\s+', '_', clean_feature).lower()

# Rename columns of the train DataFrame using the defined function
train.columns = [rename_feature(feature) for feature in train.columns]


In [61]:
train.head()

,temperature,ratio_of_epoxy_resin_to_diluent,viscosity,epoxy_resinbisphenol_a,epoxy_resinester_cyclic,epoxy_resinphenolic,additional_group_of_epoxy_resinamyl_phenol_acid_,additional_group_of_epoxy_resinh,additional_group_of_epoxy_resinacrylate,additional_group_of_epoxy_resinacrylate_,additional_group_of_epoxy_resinmethacrylate,diluentfuran_acetone,diluentothers,diluentacrylate,diluentepoxy_oil,diluentester,diluentglycidyl_ether,diluentphenol,diluentphenolic,diluentpolysiloxane,diluentsiloxane_,additional_group_of_diluent1_4butanediol,additional_group_of_diluent1_6hexanediol,additional_group_of_diluentbisphosphonatepiperazine_hydroxyl,additional_group_of_diluentcardanol_was_grafted_with_silicone,additional_group_of_diluentdipropylene_glycol,additional_group_of_diluentepoxy_ricin_acid,additional_group_of_diluenth,additional_group_of_diluentparsley_methyl_group,additional_group_of_diluentpolyoxycardanol,additional_group_of_diluentanacardol_,additional_group_of_diluentbenzyl_alcohol,additional_group_of_diluentbutyl_,additional_group_of_diluentcyclohexene_oxide,additional_group_of_diluentdiglycol,additional_group_of_diluentepoxy_group,additional_group_of_diluenteugenol,additional_group_of_diluentheterocycle,additional_group_of_diluentmethylene,additional_group_of_diluentmethyl_,additional_group_of_diluentnbutyl,additional_group_of_diluentnonyl,additional_group_of_diluentoxylene,additional_group_of_diluentoctyl_group_,additional_group_of_diluentpaminophenol,additional_group_of_diluentphenyl,additional_group_of_diluentphenylethylene_oxide,additional_group_of_diluentphosphoric_acid,additional_group_of_diluentpropylene_glycol,additional_group_of_diluentstyrene_,additional_group_of_diluentthymol
0,29.795191,95：5,1243.068262,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,29.736358,90：10,597.888811,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,29.731920,85：15,368.191414,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,29.729081,80：20,221.252639,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,34.840280,95：5,776.917664,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [62]:
def ratio_to_float(ratio_string):
    num, denom = map(float, ratio_string.split('：'))
    return num / (num + denom)

# Apply the function to the column
train['ratio_of_epoxy_resin_to_diluent'] = train['ratio_of_epoxy_resin_to_diluent'].apply(ratio_to_float)

In [64]:
# Get the list of column names
columns = train.columns.tolist()

# Move 'Viscosity' to the end
columns.remove('viscosity')
columns.append('viscosity')

# Reorder the DataFrame
train = train[columns]


In [65]:
train.head()

,temperature,ratio_of_epoxy_resin_to_diluent,epoxy_resinbisphenol_a,epoxy_resinester_cyclic,epoxy_resinphenolic,additional_group_of_epoxy_resinamyl_phenol_acid_,additional_group_of_epoxy_resinh,additional_group_of_epoxy_resinacrylate,additional_group_of_epoxy_resinacrylate_,additional_group_of_epoxy_resinmethacrylate,diluentfuran_acetone,diluentothers,diluentacrylate,diluentepoxy_oil,diluentester,diluentglycidyl_ether,diluentphenol,diluentphenolic,diluentpolysiloxane,diluentsiloxane_,additional_group_of_diluent1_4butanediol,additional_group_of_diluent1_6hexanediol,additional_group_of_diluentbisphosphonatepiperazine_hydroxyl,additional_group_of_diluentcardanol_was_grafted_with_silicone,additional_group_of_diluentdipropylene_glycol,additional_group_of_diluentepoxy_ricin_acid,additional_group_of_diluenth,additional_group_of_diluentparsley_methyl_group,additional_group_of_diluentpolyoxycardanol,additional_group_of_diluentanacardol_,additional_group_of_diluentbenzyl_alcohol,additional_group_of_diluentbutyl_,additional_group_of_diluentcyclohexene_oxide,additional_group_of_diluentdiglycol,additional_group_of_diluentepoxy_group,additional_group_of_diluenteugenol,additional_group_of_diluentheterocycle,additional_group_of_diluentmethylene,additional_group_of_diluentmethyl_,additional_group_of_diluentnbutyl,additional_group_of_diluentnonyl,additional_group_of_diluentoxylene,additional_group_of_diluentoctyl_group_,additional_group_of_diluentpaminophenol,additional_group_of_diluentphenyl,additional_group_of_diluentphenylethylene_oxide,additional_group_of_diluentphosphoric_acid,additional_group_of_diluentpropylene_glycol,additional_group_of_diluentstyrene_,additional_group_of_diluentthymol,viscosity
0,29.795191,0.95,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,1243.068262
1,29.736358,0.90,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,597.888811
2,29.731920,0.85,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,368.191414
3,29.729081,0.80,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,221.252639
4,34.840280,0.95,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,776.917664


In [66]:
# Convert boolean columns to integers (1 for True, 0 for False)
train = train * 1


In [67]:
train.head()

,temperature,ratio_of_epoxy_resin_to_diluent,epoxy_resinbisphenol_a,epoxy_resinester_cyclic,epoxy_resinphenolic,additional_group_of_epoxy_resinamyl_phenol_acid_,additional_group_of_epoxy_resinh,additional_group_of_epoxy_resinacrylate,additional_group_of_epoxy_resinacrylate_,additional_group_of_epoxy_resinmethacrylate,diluentfuran_acetone,diluentothers,diluentacrylate,diluentepoxy_oil,diluentester,diluentglycidyl_ether,diluentphenol,diluentphenolic,diluentpolysiloxane,diluentsiloxane_,additional_group_of_diluent1_4butanediol,additional_group_of_diluent1_6hexanediol,additional_group_of_diluentbisphosphonatepiperazine_hydroxyl,additional_group_of_diluentcardanol_was_grafted_with_silicone,additional_group_of_diluentdipropylene_glycol,additional_group_of_diluentepoxy_ricin_acid,additional_group_of_diluenth,additional_group_of_diluentparsley_methyl_group,additional_group_of_diluentpolyoxycardanol,additional_group_of_diluentanacardol_,additional_group_of_diluentbenzyl_alcohol,additional_group_of_diluentbutyl_,additional_group_of_diluentcyclohexene_oxide,additional_group_of_diluentdiglycol,additional_group_of_diluentepoxy_group,additional_group_of_diluenteugenol,additional_group_of_diluentheterocycle,additional_group_of_diluentmethylene,additional_group_of_diluentmethyl_,additional_group_of_diluentnbutyl,additional_group_of_diluentnonyl,additional_group_of_diluentoxylene,additional_group_of_diluentoctyl_group_,additional_group_of_diluentpaminophenol,additional_group_of_diluentphenyl,additional_group_of_diluentphenylethylene_oxide,additional_group_of_diluentphosphoric_acid,additional_group_of_diluentpropylene_glycol,additional_group_of_diluentstyrene_,additional_group_of_diluentthymol,viscosity
0,29.795191,0.95,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1243.068262
1,29.736358,0.90,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,597.888811
2,29.731920,0.85,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,368.191414
3,29.729081,0.80,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,221.252639
4,34.840280,0.95,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,776.917664


In [26]:
original_features = list(train.columns)
len(original_features)

51

In [27]:
train.head()

,temperature,ratio,epoxy_resin_bisphenol_a,epoxy_resin_ester_cyclic,epoxy_resin_phenolic,Additional Group of Epoxy resin_Amyl phenol acid,additional_group_epoxy_resin_h,additional_group_epoxy_resin_acrylate,Additional Group of Epoxy resin_acrylate,additional_group_epoxy_resin_methacrylate,diluent_furan_acetone,diluent_others,diluent_acrylate,diluent_epoxy_oil,diluent_ester,Diluent_glycidyl ether,diluent_phenol,diluent_phenolic,diluent_polysiloxane,Diluent_siloxane,additional_group_diluent_1_4_butanediol,additional_group_diluent_1_6_hexanediol,additional_group_diluent_bisphosphonate_piperazine_hydroxyl,additional_group_diluent_cardanol_silicone_grafted,additional_group_diluent_dipropylene_glycol,additional_group_diluent_epoxy_ricin_acid,additional_group_diluent_h,additional_group_diluent_parsley_methyl_group,additional_group_diluent_polyoxycardanol,Additional Group of Diluent_anacardol,Additional Group of Diluent_benzyl alcohol,Additional Group of Diluent_butyl,Additional Group of Diluent_cyclohexene oxide,additional_group_diluent_diglycol,additional_group_diluent_epoxy_group,additional_group_diluent_eugenol,additional_group_diluent_heterocycle,additional_group_diluent_methylene,Additional Group of Diluent_methyl,additional_group_diluent_n_butyl,additional_group_diluent_nonyl,additional_group_diluent_o_xylene,Additional Group of Diluent_octyl group,additional_group_diluent_p_aminophenol,additional_group_diluent_phenyl,Additional Group of Diluent_phenylethylene oxide,Additional Group of Diluent_phosphoric acid,Additional Group of Diluent_propylene glycol,Additional Group of Diluent_styrene,additional_group_diluent_thymol,Viscosity
0,29.795191,0.95,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1243.068262
1,29.736358,0.90,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,597.888811
2,29.731920,0.85,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,368.191414
3,29.729081,0.80,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,221.252639
4,34.840280,0.95,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,776.917664


In [68]:
target='viscosity'
features = train.columns.drop(target)


In [73]:
features

Index(['temperature', 'ratio_of_epoxy_resin_to_diluent',
       'epoxy_resinbisphenol_a', 'epoxy_resinester_cyclic',
       'epoxy_resinphenolic',
       'additional_group_of_epoxy_resinamyl_phenol_acid_',
       'additional_group_of_epoxy_resinh',
       'additional_group_of_epoxy_resinacrylate',
       'additional_group_of_epoxy_resinacrylate_',
       'additional_group_of_epoxy_resinmethacrylate', 'diluentfuran_acetone',
       'diluentothers', 'diluentacrylate', 'diluentepoxy_oil', 'diluentester',
       'diluentglycidyl_ether', 'diluentphenol', 'diluentphenolic',
       'diluentpolysiloxane', 'diluentsiloxane_',
       'additional_group_of_diluent1_4butanediol',
       'additional_group_of_diluent1_6hexanediol',
       'additional_group_of_diluentbisphosphonatepiperazine_hydroxyl',
       'additional_group_of_diluentcardanol_was_grafted_with_silicone',
       'additional_group_of_diluentdipropylene_glycol',
       'additional_group_of_diluentepoxy_ricin_acid',
       'additional_g

In [70]:
modeli = RandomForestRegressor(n_estimators=300, min_samples_leaf=30, random_state=1)
modeli.fit(train[features], train['viscosity'])

RandomForestRegressor(min_samples_leaf=30, n_estimators=300, random_state=1)

In [71]:
import pickle
# open a file, where you ant to store the data
file = open('modelforepoxy.pkl', 'wb')

# dump information to that file
pickle.dump(modeli, file)

In [72]:
model = open('modelforepoxy.pkl','rb')
forest = pickle.load(model)